# 패키지 불러오기

In [ ]:
import sys
import sqlite3
import time
import datetime
from pytz import timezone

import telegram
from telegram import message
from telegram.ext import Updater
from telegram.ext import MessageHandler, Filters

import smtplib
from email.message import EmailMessage
import getpass

# 텔레그램 및 이메일 계정 연동

In [ ]:
password = '********' # 이메일 비밀번호
token = '1931133170:AAHJv6liRRrYt4ZgHN***********'
id = "194472****"

# 답변 분류 및 생성 모델 로드

In [ ]:
# 고객 리뷰 분류
def review_classification(review):
    results = review_classifier(review)
    id2label = {"LABEL_0": "칭찬", "LABEL_1": "APP 버그", "LABEL_2": "정정 요청", "LABEL_3": "기타"}
    
    for idx, result in enumerate(results):
        for k, v in result.items():
            if k == 'label':
                catagory_ = id2label[v]
            else:
                score_ = v
                
    return catagory_, score_

# 답변 생성
def reply_genertation(str_):
    tokenized_indexs = tokenizer.encode(str_)
    input_ids = torch.tensor([tokenizer.bos_token_id,]  + tokenized_indexs +[tokenizer.eos_token_id]).unsqueeze(0)
    sample_output = model.generate(input_ids=input_ids)
    ans = tokenizer.decode(sample_output[0].tolist()[len(tokenized_indexs)+1:],skip_special_tokens=True)
    ans = re.findall(r'[가-힣| |\U00010000-\U0010ffff]+\?|[가-힣| |\U00010000-\U0010ffff]+\.', ans)
    ans = ans[:3]
    answer = ''
    for i in range(3):
        answer += ans[i]
        
    return answer

# 챗봇

In [ ]:
bot = telegram.Bot(token)
bot.sendMessage(chat_id=id, text="안녕하세요! 고객님 리브버트입니다😊 무엇을 도와드릴까요?")

# updater
updater = Updater(token=token, use_context=True)
dispatcher = updater.dispatcher
updater.start_polling()

# 날짜 추출 및 이메일 양식 생성
def printDate():
    tn= datetime.datetime.now(timezone('Asia/Seoul'))
    weekDay = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    
    return tn.strftime("%Y년 %m월 %d일 ") + str(weekDay[tn.weekday()])


chating_data = '“Customer” 고객님과 아래와 같은 대화를 진행했습니다.\n'
chating_data += '\n\n\n"대화 원본"\n'
date = printDate()
chating_data += '-------------------------' + date + '-------------------------\n'

cata_ = ''
def handler(update, context):
    global chating_data, inp, cata_
    user_text = update.message.text
    
    if str( user_text ) == '없습니다.':
        time_now = datetime.datetime.now(timezone('Asia/Seoul'))
        time_now_str = str(time_now)[11:19]
        cus_chat = '[고객][' + time_now_str + ']' + user_text
        chating_data += cus_chat
        chating_data += '\n\n\n - KB_리브버트 드림'
        
        chat_time = datetime.datetime.now(timezone('Asia/Seoul'))
        chat_time_str = str(chat_time)[11:19]
        
        conn = sqlite3.connect("test.db")
        cur = conn.cursor()
        cur.execute("INSERT INTO voc VALUES (?,?)", [chating_data, chat_time_str])
        conn.commit()
        
        tomail = ''
        for row in cur.execute("SELECT email FROM employee WHERE team = 'develop'"):
            tomail += row[0] + ', '
            
        message = EmailMessage()
        message['Subject'] = '[고객의견]' +'[' + str(cata_) + ']' + '고객 의견을 전달합니다.' #str(cata_) 있던 곳
        message['From'] = 'kbalbert_chatbot@naver.com'
        message['To'] = tomail
        message.set_content(str(chating_data))

        with smtplib.SMTP_SSL('smtp.naver.com', 465) as server:
            server.ehlo()
            server.login('kbalbert_chatbot', password)
            server.send_message(message)
        print('이메일 전송이 완료됬습니다.')
        sys.exit()
    
    # 의견 분류
    c, s = review_classification(str(user_text))
    if len( cata_ ) == 0:
        cata_ = c
    time_now = datetime.datetime.now(timezone('Asia/Seoul'))
    time_now_str = str(time_now)[11:19]
    cus_chat = '[고객][' + time_now_str + ']' + user_text
    
    # 답변 생성
    text_ = reply_genertation(str(user_text))
    bot.send_message(chat_id=id, text=str(text_))
    time_now = datetime.datetime.now(timezone('Asia/Seoul'))
    time_now_str = str(time_now)[11:19]
    bot_chat = '[챗봇][' + time_now_str + ']' + str(text_)
    
    # 분류 데이터 DB 저장
    idx = len(df)
    at = str(time_now)[:19]
    content = user_text

    if cata_ == '칭찬':
        catagory = 0
    elif cata_ == 'APP 버그':
        catagory = 1
    elif cata_ == '정정 요청':
        catagory = 2
    else:
        catagory = 3
        
    row = [idx, at, content, catagory]
    df.loc[len(df)] = row
    df.to_csv("Review_Label_data.csv", index=False)
    
    chating_data += cus_chat + '\n' + bot_chat + '\n'
echo_handler = MessageHandler(Filters.text, handler)
dispatcher.add_handler(echo_handler)